In [1]:
# Langchain dependencies
import streamlit as st
import os
from langchain.document_loaders.pdf import PyPDFDirectoryLoader # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importing text splitter from Langchain
from langchain.embeddings import OpenAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document # Importing Document schema from Langchain
from langchain.vectorstores.chroma import Chroma # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI # Import OpenAI LLM
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations

In [2]:
#Setup Page Configuration for Streamlit
st.set_page_config(
    page_title = "Local RAG llm",
    layout='wide')

In [3]:
# Directory to your pdf files:
DATA_PATH = "../data"
def load_documents():
  """
  Load PDF documents from the specified directory using PyPDFDirectoryLoader.
  Returns:
  List of Document objects: Loaded PDF documents represented as Langchain
                                                          Document objects.
  """
  # Initialize PDF loader with specified directory
  document_loader = PyPDFDirectoryLoader(DATA_PATH) 
  # Load PDF documents and return them as a list of Document objects
  return document_loader.load() 

documents = load_documents() # Call the function
# Inspect the contents of the first document as well as metadata
print(documents[0])

page_content='Traditional System of Medicine  
  
Pharmacognosy has been basically evolved as an applied science 
pertaining to the study of all types of drugs of natural origin. However, its 
subject matter is directed towards the modern allopathic medicine. During 
the course of developments, many civiliza tions have raised and perished 
but the systems of medicines developed by them in various parts of the 
world are still practised, and are also popular as the alternative systems of 
medicine. These are the alternative systems in the sense that modern 
allopat hic system has been globally acclaimed as the principal system of 
medicine, and so all the other systems prevalent and practised in various 
parts of the world are supposed to be alternative systems. The philosophy 
and the basic principles of these so calle d alternative systems might differ 
significantly from each other, but the fact cannot be denied that these 
systems have served the humanity for the treatment and manag

In [4]:
def split_text(documents: list[Document]):
  """
  Split the text content of the given list of Document objects into smaller chunks.
  Args:
    documents (list[Document]): List of Document objects containing text content to split.
  Returns:
    list[Document]: List of Document objects representing the split text chunks.
  """
  # Initialize text splitter with specified parameters
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, # Size of each chunk in characters
    chunk_overlap=100, # Overlap between consecutive chunks
    length_function=len, # Function to compute the length of the text
    add_start_index=True, # Flag to add start index to each chunk
  )

  # Split documents into smaller chunks using text splitter
  chunks = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

  # Print example of page content and metadata for a chunk
  document = chunks[0]
  print(document.page_content)
  print(document.metadata)

  return chunks # Return the list of split text chunks

In [5]:
# service_context=ServiceContext.from_defaults(llm=OpenAI(temperature=0, model="gpt-3.5-turbo", api_key="sk-proj-O7P7AxMfq3M13eMHz5ZxT3BlbkFJzngJzu9d4UFdBJlMjp2x"))
# print(service_context.llm.complete("Hello!"))


# Path to the directory to save Chroma database
CHROMA_PATH = "chroma"
def save_to_chroma(chunks: list[Document]):
  """
  Save the given list of Document objects to a Chroma database.
  Args:
  chunks (list[Document]): List of Document objects representing text chunks to save.
  Returns:
  None
  """

  # Clear out the existing database directory if it exists
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

  # Create a new Chroma database from the documents using OpenAI embeddings
  db = Chroma.from_documents(
    chunks,
    OpenAIEmbeddings(),
    persist_directory=CHROMA_PATH
  )

  # Persist the database to disk
  db.persist()
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [6]:
def generate_data_store():
  """
  Function to generate vector database in chroma from documents.
  """
  documents = load_documents() # Load documents from a source
  chunks = split_text(documents) # Split documents into manageable chunks
  #save_to_chroma(chunks) # Save the processed data to a data store

# Load environment variables from a .env file
load_dotenv()
# Generate the data store
generate_data_store()

Split 42 documents into 544 chunks.
Traditional System of Medicine  
  
Pharmacognosy has been basically evolved as an applied science 
pertaining to the study of all types of drugs of natural origin. However, its 
subject matter is directed towards the modern allopathic medicine. During
{'source': '../data/Traditional_System_of_Medicine.pdf', 'page': 0, 'start_index': 0}


In [7]:
query_text = input()

In [ ]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

In [ ]:
def query_rag(query_text):
  """
  Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
  Args:
    - query_text (str): The text to query the RAG system with.
  Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
  """
  # YOU MUST - Use same embedding function as before
  embedding_function = OpenAIEmbeddings()

  # Prepare the database
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
  
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=3)

  # Check if there are any matching results or if the relevance score is too low
  if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)
  
  # Initialize OpenAI chat model
  model = ChatOpenAI()

  # Generate response text based on the prompt
  response_text = model.predict(prompt)
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources}"
  return formatted_response, response_text

# Let's call our function we have defined
formatted_response, response_text = query_rag(query_text)
# and finally, inspect our final response!
print(response_text)

Unable to find matching results.
Without additional context, it is difficult to provide a specific answer to this question. However, some potential negative side effects of traditional medicine could include adverse reactions to herbal ingredients, lack of regulation leading to inconsistent quality and potency of products, and potential interactions with other medications.


In [ ]:
#The RAG is working